# Try reading gnomAD data into memory

In [4]:
using Revise
using gnomAD

┌ Info: Precompiling gnomAD [a5d00db5-5f0a-49b9-aaff-6f4121a13ee1]
└ @ Base loading.jl:1423


In [5]:
data = "/Users/biona001/.julia/dev/gnomAD/data/gnomad.genomes.r2.1.1.nfe.common.adj.ld.bm"
bm = hail_block_matrix(data);

In [6]:
@time begin
    out = zeros(10, 10)
    for i in 1:10, j in 1:10
        out[i, j] = bm[i, j]
    end
end
out

2023-03-01 16:39:40.545 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
 21.392457 seconds (11.79 k allocations: 673.166 KiB, 0.09% compilation time)


Initializing Hail with default parameters...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://dn52ecqn.sunet:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /Users/biona001/.julia/dev/gnomAD/test/hail-20230301-1639-0.2.109-b71b065e4bb6.log


10×10 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0179389   -0.00994853
 0.0  1.0        -0.00041394  0.00540477      0.00211069   0.00171555
 0.0  0.0         1.0         0.0177243       0.0         -0.000284646
 0.0  0.0         0.0         1.0             0.00627614   0.00131174
 0.0  0.0         0.0         0.0            -0.0045831    0.00119736
 0.0  0.0         0.0         0.0         …   0.0145138    0.00530853
 0.0  0.0         0.0         0.0             0.00089687   0.00328037
 0.0  0.0         0.0         0.0             0.123663     0.377178
 0.0  0.0         0.0         0.0             1.0          0.00725709
 0.0  0.0         0.0         0.0             0.0          1.0

In [12]:
@time bm[1:1000, 1:500]

  0.513938 seconds (15.47 k allocations: 4.677 MiB, 2.18% compilation time)


1000×500 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.10396      -0.00442173
 0.0  1.0        -0.00041394  0.00540477      0.00714046    0.000179424
 0.0  0.0         1.0         0.0177243      -0.00255837    0.00418069
 0.0  0.0         0.0         1.0            -0.000987178   0.0
 0.0  0.0         0.0         0.0             0.0117148    -0.000825575
 0.0  0.0         0.0         0.0         …  -0.00810211    0.0
 0.0  0.0         0.0         0.0            -0.0360996    -0.00376967
 0.0  0.0         0.0         0.0             0.00271577    0.00468353
 0.0  0.0         0.0         0.0            -0.00977714    0.0032796
 0.0  0.0         0.0         0.0            -0.0106557    -0.000525512
 0.0  0.0         0.0         0.0         …  -0.0127855    -0.000778327
 0.0  0.0         0.0         0.0             0.021351      0.00109237
 0.0  0.0         0.0         0.0             0.055791     -0.00806577
 ⋮                                        ⋱                
 0

In [15]:
# indexing non-contiguous region is slow
@time bm[1, 1:2:100]

  7.688574 seconds (269 allocations: 4.719 KiB)


50-element Vector{Float64}:
  0.9999999999999998
 -0.018189111101561392
  0.008027968930178427
 -0.04215117028151553
  0.017938860182932757
 -0.0016337026399970787
  0.10486066844855962
  0.025560991521727777
 -0.010546800431644462
 -0.03985325918324301
 -0.0010861133958637733
  0.0007699733072654338
  0.001359820828733087
  ⋮
  0.03061218870032942
 -0.0036389897468388677
  0.0070613701455110875
  0.010107017312441766
  0.009773613328151184
 -0.0006180327700999952
 -0.007792310263082733
 -0.001076654549804239
  0.009592268611895462
  0.008936790685499503
  0.014353376620577216
  0.017235755053720745

In [44]:
using PyCall
py"""
from hail.linalg import BlockMatrix
import numpy as np
def convert(bm, range_start, range_end):
    return bm[range_start:range_end, range_start:range_end].to_numpy()
"""
@time py"convert"(bm.bm, 0, 100)

  0.827913 seconds (33.87 k allocations: 1.972 MiB, 1.93% compilation time)


100×100 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0172358    -0.00857776
 0.0  1.0        -0.00041394  0.00540477     -0.0188801     0.05925
 0.0  0.0         1.0         0.0177243      -0.0277042    -0.000268512
 0.0  0.0         0.0         1.0            -0.000468342  -0.000966708
 0.0  0.0         0.0         0.0            -7.47553e-5   -0.00298802
 0.0  0.0         0.0         0.0         …   0.00207735    0.00152017
 0.0  0.0         0.0         0.0             0.00256425    0.00136302
 0.0  0.0         0.0         0.0             0.0137416    -0.000680087
 0.0  0.0         0.0         0.0            -0.020671      0.000342287
 0.0  0.0         0.0         0.0             0.00195264   -0.00451867
 0.0  0.0         0.0         0.0         …   0.0018285    -0.00406988
 0.0  0.0         0.0         0.0             0.0152337     0.0100539
 0.0  0.0         0.0         0.0             0.0193       -0.0136828
 ⋮                                        ⋱          

### Python code

In [ ]:
from hail.linalg import BlockMatrix
import numpy as np
bm_files = "gnomad.genomes.r2.1.1.nfe.common.adj.ld.bm"
bm = BlockMatrix.read(bm_files)

# get a slice fast
x = bm[0:100, 0:100]
X = x.to_numpy()
X

In [ ]:
def convert(bm, range1, range2)
    return bm[range1, range2].to_numpy()